In [455]:
import struct
import numpy as np
import intvalpy as ip
from functools import cmp_to_key

In [456]:
def read_bin_file_with_numpy(file_path):
  with open(file_path, 'rb') as f:
    header_data = f.read(256)
    side, mode, frame_count = struct.unpack('<BBH', header_data[:4])

    frames = []
    point_dtype = np.dtype('<8H')

    for _ in range(frame_count):
      frame_header_data = f.read(16)
      stop_point, timestamp = struct.unpack('<HL', frame_header_data[:6])
      frame_data = np.frombuffer(f.read(1024 * 16), dtype=point_dtype)
      frames.append(frame_data)

    return np.array(frames)

In [457]:
def convert_to_voltage(data):
  return data / 16384.0 - 0.5

In [458]:
def are_intersected(x, y):
  sup = y.a if x.a < y.a else x.a
  inf = x.b if x.b < y.b else y.b
  return sup - inf <= 1e-15

In [459]:
def are_adjusted_to_each_other(x, y):
  return x.b == y.a or y.b == x.a

In [460]:
def merge_intervals(x, y):
  return ip.Interval(min(x.a, y.a), max(x.b, y.b))

In [461]:
def mode(x):
  if len(x) == 0:
    return []

  edges = set()
  for x_i in x:
    edges.add(x_i.a)
    edges.add(x_i.b)
  edges = sorted(list(edges))

  z = []
  for index, a in enumerate(edges):
    b = a if index == len(edges) - 1 else edges[index + 1]
    z.append(ip.Interval(a, b))

  mu = []
  for z_i in z:
    mu_i = 0
    for x_i in x:
      if z_i in x_i:
        mu_i += 1
    mu.append(mu_i)

  max_mu = max(mu)
  K = [index for index, element in enumerate(mu) if element == max_mu]

  m = [z[k] for k in K]
  merged_m = []

  current_interval = m[0]

  for next_interval in m[1:]:
    if are_intersected(current_interval, next_interval) or are_adjusted_to_each_other(current_interval, next_interval):
      current_interval = merge_intervals(current_interval, next_interval)
    else:
      merged_m.append(current_interval)
      current_interval = next_interval

  merged_m.append(current_interval)

  if len(merged_m) == 1:
    return merged_m[0]

  return merged_m

In [462]:
def med_k(x):
  starts = [float(interval.a) for interval in x]
  ends = [float(interval.b) for interval in x]
  return ip.Interval(np.median(starts), np.median(ends))

In [463]:
def med_p(x):
  x = sorted(x, key=cmp_to_key(lambda x, y: (x.a + x.b) / 2 - (y.a + y.b) / 2))
  print(x)

  mid = len(x) // 2

  if len(x) % 2 == 0:
    return (x[mid - 1] + x[mid]) / 2

  return x[mid]

In [469]:
def scalar_to_interval(x, rad):
  return ip.Interval(x - rad, x + rad)
scalar_to_interval_vec = np.vectorize(scalar_to_interval)

In [464]:
x_data = read_bin_file_with_numpy('-0.205_Ivl_side_a_fast_data.bin')
y_data = read_bin_file_with_numpy('0.225_ Ivl_side_a_fast_data.bin')

In [465]:
x_voltage = convert_to_voltage(x_data)
y_voltage = convert_to_voltage(y_data)

In [470]:
x_voltage[0][0][0]

-0.18609619140625

In [ ]:
rad = 2 ** -14
x_voltage_int = scalar_to_interval_vec(x_voltage, rad)
y_voltage_int = scalar_to_interval_vec(y_voltage, rad)

ValueError: could not convert string to float: "'[-0.186157, -0.186035]'"

In [474]:
rad

6.103515625e-05